In [6]:
# Package import
from modopt.math.metrics import ssim
from modopt.opt.linear import Identity
from modopt.opt.proximity import SparseThreshold
from mri.operators import FFT, WaveletN, WaveletUD2, NonCartesianFFT
from mri.operators.utils import convert_mask_to_locations
from mri.reconstructors import SingleChannelReconstructor
from pysap.data import get_sample_data
import pysap

# Third party import
import numpy as np
import matplotlib.pyplot as plt

In [40]:
image = get_sample_data('2d-mri')

#Radial mask
radial_mask = get_sample_data("mri-radial-samples")
radial_kspace_loc = radial_mask.data

#Cartesian mask
mask = get_sample_data("cartesian-mri-mask")
kspace_loc = convert_mask_to_locations(mask.data)

In [74]:
#Radial mask fourier
#fourier_op = NonCartesianFFT(samples=radial_kspace_loc, shape=image.shape,implementation='cpu')

#Cartesian mask fourier
fourier_op = FFT(samples=kspace_loc, shape=image.shape)

kspace_data = fourier_op.op(image.data)
print(np.mean(np.abs(kspace_data)))
kspace_data += 1e-7*np.random.randn(2*len(kspace_data)).view(np.complex128)

2.438104974074126e-07


/home/guillaume/temp/pysap-mri/mri/operators/fourier/utils.py:78: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask[test] = 1


Take epsilon~1e-7 to get good results with cartesian mask

In [75]:
linear_op = WaveletUD2(wavelet_id=24,nb_scale=3,)
#linear_op = WaveletN(wavelet_name="db8", nb_scales=4)

epsilon = 1e-7
regularizer_op = SparseThreshold(linear_op, epsilon, thresh_type="soft")
reconstructor = SingleChannelReconstructor(
    fourier_op=fourier_op,
    linear_op=linear_op,
    regularizer_op=regularizer_op,
    gradient_formulation='analysis',
    verbose=1,
)
x_final, costs, metrics = reconstructor.reconstruct(
    kspace_data=kspace_data,
    optimization_alg='condatvu',
    num_iterations=100,
)

Lipschitz constant is 1.1
The lipschitz constraint is satisfied


 - mu:  1e-07
 - lipschitz constant:  1.1
 - tau:  0.9383364158273717
 - sigma:  0.5
 - rho:  1.0
 - std:  None
 - 1/tau - sigma||L||^2 >= beta/2:  True
 - data:  (512, 512)
 - wavelet:  <mri.operators.linear.wavelet.WaveletUD2 object at 0x7f5b68868eb8> - 3
 - max iterations:  100
 - number of reweights:  0
 - primal variable shape:  (512, 512)
 - dual variable shape:  (1835008,)
----------------------------------------
Starting optimization...


100% (100 of 100) |######################| Elapsed Time: 0:01:02 Time:  0:01:02


 - final iteration number:  100
 - final cost value:  1.0389879493094792e-07
 - converged:  True
Done.
Execution time:  255.59949499999993  seconds
----------------------------------------


In [76]:
print(ssim(np.abs(x_final),image))
pysap.Image(data=x_final).show()

0.9383002984839741
